In [7]:
import getpass
import os

SNOWFLAKE_ACCOUNT = ""
SNOWFLAKE_USERNAME = ""
SNOWFLAKE_PASSWORD = ""
SNOWFLAKE_DATABASE = ""
SNOWFLAKE_SCHEMA = ""
SNOWFLAKE_WAREHOUSE = ""
SNOWFLAKE_ROLE = ""


In [9]:
print(f"snowflake://{SNOWFLAKE_USERNAME}:{SNOWFLAKE_PASSWORD}@{SNOWFLAKE_ACCOUNT}/{SNOWFLAKE_DATABASE}/{SNOWFLAKE_SCHEMA}?warehouse={SNOWFLAKE_WAREHOUSE}&role={SNOWFLAKE_ROLE}")

snowflake://:@//?warehouse=&role=


In [119]:
import snowflake.sqlalchemy

In [121]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase

# Create a SQLDatabase instance
db = SQLDatabase.from_uri(f"snowflake://{SNOWFLAKE_USERNAME}:{SNOWFLAKE_PASSWORD}@{SNOWFLAKE_ACCOUNT}/{SNOWFLAKE_DATABASE}/{SNOWFLAKE_SCHEMA}?warehouse={SNOWFLAKE_WAREHOUSE}&role={SNOWFLAKE_ROLE}")



In [122]:
from langchain_openai import ChatOpenAI  

In [123]:
OPENAI_API_KEY =  ''



llm = ChatOpenAI(model="gpt-4",api_key=OPENAI_API_KEY)


In [127]:
# Initialize the SQL agent toolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)  # Replace `llm` with your language model instance

# Get the tools for the agent
tools = toolkit.get_tools()


In [63]:
db.run("SELECT * FROM EDW.github_notebook_cells LIMIT 10;")

'[(18201, \'langgraph/docs/docs/tutorials/introduction.ipynb\', \'introduction.ipynb\', \'tutorials\', 2, \'%%capture --no-stderr\\n%pip install -U langgraph langsmith langchain_anthropic\', \'# 🚀 LangGraph Quick Start\\n\\nIn this tutorial, we will build a support chatbot in LangGraph that can:\\n\\n✅ **Answer common questions** by searching the web  \\n✅ **Maintain conversation state** across calls  \\n✅ **Route complex queries** to a human for review  \\n✅ **Use custom state** to control its...\', \'Markdown not found\', datetime.datetime(2024, 11, 29, 10, 33, 8, 205000)), (18202, \'langgraph/docs/docs/tutorials/introduction.ipynb\', \'introduction.ipynb\', \'tutorials\', 3, \'import getpass\\nimport os\\n\\n\\ndef _set_env(var: str):\\n    if not os.environ.get(var):\\n        os.environ[var] = getpass.getpass(f"{var}: ")\\n\\n\\n_set_env("ANTHROPIC_API_KEY")\', \'Markdown not found\', \'Set up LangSmith for LangGraph development\\n\\n        Sign up for LangSmith to quickly spot i

In [65]:
db.get_usable_table_names()

['github_classes',
 'github_consolidated_notebooks',
 'github_functions',
 'github_global_statements',
 'github_markdown_docs',
 'github_markdown_sections',
 'github_notebook_cells',
 'userdetails']

In [129]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType


In [139]:
custom_prompt = "You are a SQL expert. Given a user's question, create a syntactically correct SQL query to retrieve the necessary data from the database. The database contains various tables with various functions, classes and methods. Go over the code column in each table. If the user asks specfic functions/classes/code retreive the full code to the user"


In [141]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [143]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    state_modifier=custom_prompt
)

In [147]:
agent_executor.run(
    "Give full code to build class StateGraph"
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


github_classes, github_consolidated_notebooks, github_functions, github_global_statements, github_markdown_docs, github_markdown_sections, github_notebook_cells, userdetails
Invoking: `sql_db_schema` with `{'table_names': 'github_classes'}`
responded: The relevant table for this query seems to be the 'github_classes'. Let's look at its schema.


CREATE TABLE github_classes (
	class_id DECIMAL(38, 0) NOT NULL IDENTITY(1,1), 
	class_name VARCHAR(255), 
	filepath VARCHAR(500), 
	filename VARCHAR(255), 
	code VARCHAR(16777216), 
	created_at TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(), 
	CONSTRAINT "SYS_CONSTRAINT_f3d96bba-efc2-4255-b54d-c0d108041274" PRIMARY KEY (class_id)
)

/*
3 rows from github_classes table:
class_id	class_name	filepath	filename	code	created_at
3401	AddableUpdatesDict	langgraph/libs/langgraph/langgraph/pregel/io.py	io.py	class AddableUpdatesDict:
def __add__(self, other: dict[str, An

"Here is the full code for the 'StateGraph' class:\n\n```python\nclass StateGraph:\n    def __init__(self, state_schema: Optional[Type[Any]]=None, config_schema: Optional[Type[Any]]=None, *, input: Optional[Type[Any]]=None, output: Optional[Type[Any]]=None) -> None:\n        super().__init__()\n        if state_schema is None:\n            if input is None or output is None:\n   ...\n```\n\nThis was found in the file 'state.py' located at 'langgraph/libs/langgraph/langgraph/graph/state.py'."